In [29]:
import tensorflow as tf
import numpy as np

In [30]:
movie_dim = 6 #3706
hidden_dim = 2 #500
num_classes = 5 #possible_ratings,eg:1,2,3,4,5
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

time_stamp_exp_lambda = 0.1

X = tf.placeholder("float32",[None,movie_dim,num_classes])
Y = tf.placeholder("float32",[None, 2])

In [31]:
weights = {

    'W': tf.Variable(tf.random_normal([movie_dim, num_classes,hidden_dim])),
    'Output_W' : tf.Variable(tf.random_normal([hidden_dim,movie_dim,num_classes]))
}

bias = {
    'b_hidden':tf.Variable(tf.random_normal([hidden_dim])),
    'b_output':tf.Variable(tf.random_normal([movie_dim, num_classes]))
}

In [32]:
def time_stamp_function_exp(dif_time_stamp,time_transform_parameter):
    return np.exp(0-abs(dif_time_stamp * time_transform_parameter))

In [33]:
def data_transform(X, Y, time_transform_parameter):
    get_ratings = X[:,:,0,0]
    target_time_stamp = np.sum(X[:,:,1,1], axis = 1)
    num_batch = np.size(X,0)
    ratings_X = np.zeros((num_batch,movie_dim,num_classes)) #num_classes:0,1,2,3,4 -> ratings:1,2,3,4,5
    for iter_num_batch in range(num_batch):
        for iter_movie in range(movie_dim):
            if get_ratings[iter_num_batch,iter_movie] > 0:
                ratings_X[iter_num_batch,iter_movie,int(get_ratings[iter_num_batch,iter_movie] - 1)] = time_stamp_function_exp(X[iter_num_batch,iter_movie,1,0] - target_time_stamp[iter_num_batch],time_transform_parameter)
    ratings_Y = tf.one_hot(Y[:,0], depth=5)
    index_Y = Y[:,1]
    return ratings_X, ratings_Y, index_Y

In [37]:
#dim(rating_X) = batch_size * movie_dim * num_classes
def time_stamp_net(rating_X, index_Y):
    #dim(layer1) = batch_size * hidden_dim
    layer1 = tf.tanh(tf.add(bias['b_hidden'],tf.tensordot(rating_X,weights['W'], axes=[[1,2],[0,1]])))
    #dim(output_layer) = batch_size * num_classes
    output_layer = tf.add(bias['b_output'], tf.tensordot(layer1, weights['Output_W'][:,index_Y,:], axes=[[1],[0]]))
    
    return output_layer

In [39]:
ratings_X, ratings_Y, index_Y = data_transform(X, Y, 1234)
pre_rating = time_stamp_net(rating_X, index_Y)
# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    logits = pre_rating, labels=Y))

scores_tensor = np.repeat(np.arange(1,6), batch_size, axis=0)
pred_scores = tf.reduce_sum(scores_tensor * tf.nn.softmax(pre_rating, axis=1), axis=1)
true_scores = tf.argmax(ratings_Y, axis=1) + 1 
loss_op = tf.losses.mean_squared_error(true_scores, pred_scores)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
#correct_pred = tf.equal(tf.argmax(pre_rating, 1), true_scores)
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

ValueError: Index out of range using input dim 3; input has only 3 dims for 'strided_slice' (op: 'StridedSlice') with input shapes: [?,6,5], [4], [4], [4] and with computed input tensors: input[3] = <1 1 1 1>.

In [ ]:
temp = np.ones((1,movie_dim,2,2))
with tf.Session() as sess:
     print(sess.run(time_stamp_net(X), feed_dict={X: temp}))

In [40]:
temp = np.ones((1,movie_dim,2,2))

out_1,out_2 = data_transform(temp,time_stamp_exp_lambda)


In [25]:
A = np.random.rand(2,3)
print (A, A[:,1])

[[ 0.62150216  0.24536912  0.25819919]
 [ 0.29301923  0.91600614  0.98113509]] [ 0.24536912  0.91600614]
